In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path

# Set up paths
sys.path.insert(0, str(Path.cwd()))

# Import our modules
from scrapers.cse_scraper import CSEDataCollector
from analysis.valuations import ValuationAnalyzer
from analysis.screeners import StockScreener
from analysis.rankings import CompanyRanker, PortfolioSuggester

# Configure display
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)
plt.style.use('seaborn-v0_8-whitegrid')

print("✅ Libraries loaded successfully!")

## 1. Load Data from CSE

In [ ]:
# Initialize data collector
collector = CSEDataCollector()

# Try to load existing data first
df = collector.get_latest_data()

if df.empty:
    print("No cached data found. Run the cell below to fetch fresh data.")
else:
    print(f"✅ Loaded {len(df)} companies from cache")
    print(f"\nColumns: {list(df.columns)}")

In [ ]:
# Uncomment to fetch fresh data from CSE (takes several minutes)
# df = collector.collect_all_data()
# print(f"Fetched {len(df)} companies")

In [ ]:
# Use sample data for demonstration
# This generates realistic sample data for testing

def generate_sample_data():
    sample_companies = [
        {"symbol": "JKH.N0000", "name": "John Keells Holdings PLC", "sector": "Diversified Holdings"},
        {"symbol": "COMB.N0000", "name": "Commercial Bank of Ceylon PLC", "sector": "Banks Finance & Insurance"},
        {"symbol": "SAMP.N0000", "name": "Sampath Bank PLC", "sector": "Banks Finance & Insurance"},
        {"symbol": "HNB.N0000", "name": "Hatton National Bank PLC", "sector": "Banks Finance & Insurance"},
        {"symbol": "DIAL.N0000", "name": "Dialog Axiata PLC", "sector": "Telecommunications"},
        {"symbol": "CARG.N0000", "name": "Cargills (Ceylon) PLC", "sector": "Stores Supplies"},
        {"symbol": "NEST.N0000", "name": "Nestle Lanka PLC", "sector": "Beverage Food & Tobacco"},
        {"symbol": "CTC.N0000", "name": "Ceylon Tobacco Company PLC", "sector": "Beverage Food & Tobacco"},
        {"symbol": "HEXP.N0000", "name": "Hemas Holdings PLC", "sector": "Diversified Holdings"},
        {"symbol": "TILE.N0000", "name": "Lanka Tiles PLC", "sector": "Manufacturing"},
        {"symbol": "LOLC.N0000", "name": "LOLC Holdings PLC", "sector": "Diversified Holdings"},
        {"symbol": "SLTL.N0000", "name": "Sri Lanka Telecom PLC", "sector": "Telecommunications"},
        {"symbol": "ALLI.N0000", "name": "Alliance Finance Company PLC", "sector": "Banks Finance & Insurance"},
        {"symbol": "RICH.N0000", "name": "Richard Pieris & Company PLC", "sector": "Diversified Holdings"},
        {"symbol": "GREG.N0000", "name": "Distilleries Company of Sri Lanka", "sector": "Beverage Food & Tobacco"},
    ]
    
    np.random.seed(42)
    data = []
    for company in sample_companies:
        price = np.random.uniform(50, 500)
        eps = np.random.uniform(5, 40)
        nav = np.random.uniform(50, 300)
        
        data.append({
            **company,
            "last_traded_price": round(price, 2),
            "change_percent": round(np.random.uniform(-5, 5), 2),
            "volume": int(np.random.uniform(10000, 500000)),
            "market_cap": int(np.random.uniform(5e9, 100e9)),
            "eps": round(eps, 2),
            "pe_ratio": round(price / eps if eps > 0 else 0, 2),
            "pb_ratio": round(price / nav if nav > 0 else 0, 2),
            "nav": round(nav, 2),
            "dividend_yield": round(np.random.uniform(0, 8), 2),
            "roe": round(np.random.uniform(8, 25), 2),
            "debt_equity": round(np.random.uniform(0.1, 1.0), 2),
        })
    return pd.DataFrame(data)

if df.empty:
    df = generate_sample_data()
    print(f"✅ Generated sample data with {len(df)} companies")

df.head(10)

## 2. Market Overview

In [ ]:
# Market Statistics
print("📊 CSE MARKET OVERVIEW")
print("=" * 50)
print(f"Total Companies: {len(df)}")

if 'pe_ratio' in df.columns:
    pe_valid = df['pe_ratio'].dropna()
    pe_valid = pe_valid[pe_valid > 0]
    print(f"Average P/E Ratio: {pe_valid.mean():.2f}")
    print(f"Median P/E Ratio: {pe_valid.median():.2f}")

if 'dividend_yield' in df.columns:
    div_valid = df['dividend_yield'].dropna()
    print(f"Average Dividend Yield: {div_valid.mean():.2f}%")

if 'roe' in df.columns:
    roe_valid = df['roe'].dropna()
    print(f"Average ROE: {roe_valid.mean():.2f}%")

In [ ]:
# Sector Distribution
if 'sector' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Sector count
    sector_counts = df['sector'].value_counts()
    axes[0].barh(sector_counts.index, sector_counts.values, color='steelblue')
    axes[0].set_title('Number of Companies by Sector')
    axes[0].set_xlabel('Count')
    
    # Average P/E by sector
    if 'pe_ratio' in df.columns:
        sector_pe = df.groupby('sector')['pe_ratio'].mean().sort_values()
        axes[1].barh(sector_pe.index, sector_pe.values, color='coral')
        axes[1].set_title('Average P/E Ratio by Sector')
        axes[1].set_xlabel('P/E Ratio')
    
    plt.tight_layout()
    plt.show()

## 3. Valuation Analysis

In [ ]:
# Initialize analyzer
analyzer = ValuationAnalyzer()

# Analyze all companies
analysis_df = analyzer.analyze_all_companies(df)

# Display analysis results
print("📈 VALUATION ANALYSIS RESULTS")
print("=" * 50)

# Show valuation status distribution
if 'valuation_status' in analysis_df.columns:
    print("\nValuation Status Distribution:")
    print(analysis_df['valuation_status'].value_counts())

In [ ]:
# Merge analysis with original data
full_df = df.merge(
    analysis_df[['symbol', 'intrinsic_value_graham', 'margin_of_safety', 
                 'valuation_status', 'value_signals_count']], 
    on='symbol', 
    how='left'
)

# Show undervalued stocks
print("\n🎯 POTENTIALLY UNDERVALUED STOCKS:")
undervalued = full_df[full_df['valuation_status'].isin(['Undervalued', 'Strongly Undervalued'])]
display_cols = ['symbol', 'name', 'last_traded_price', 'pe_ratio', 'pb_ratio', 
                'intrinsic_value_graham', 'margin_of_safety', 'valuation_status']
available_cols = [c for c in display_cols if c in undervalued.columns]
undervalued[available_cols]

## 4. Stock Screening Strategies

In [ ]:
# Initialize screener
screener = StockScreener(full_df)

# Run all strategies
strategy_results = screener.run_all_strategies()

print("\n📋 SCREENING RESULTS SUMMARY:")
print("=" * 50)
for strategy, result in strategy_results.items():
    print(f"{strategy:15}: {len(result):3} stocks found")

In [ ]:
# Value Investing Screen
print("\n💎 VALUE INVESTING PICKS (Low P/E, Low P/B):")
print("=" * 50)
value_stocks = strategy_results.get('value', pd.DataFrame())
if not value_stocks.empty:
    cols = ['symbol', 'name', 'last_traded_price', 'pe_ratio', 'pb_ratio', 'eps']
    available = [c for c in cols if c in value_stocks.columns]
    display(value_stocks[available].head(10))

In [ ]:
# Dividend Investing Screen
print("\n💰 HIGH DIVIDEND YIELD STOCKS:")
print("=" * 50)
dividend_stocks = strategy_results.get('dividend', pd.DataFrame())
if not dividend_stocks.empty:
    cols = ['symbol', 'name', 'last_traded_price', 'dividend_yield', 'pe_ratio']
    available = [c for c in cols if c in dividend_stocks.columns]
    display(dividend_stocks[available].head(10))

In [ ]:
# Growth Investing Screen
print("\n🚀 GROWTH STOCKS (High ROE):")
print("=" * 50)
growth_stocks = strategy_results.get('growth', pd.DataFrame())
if not growth_stocks.empty:
    cols = ['symbol', 'name', 'last_traded_price', 'roe', 'pe_ratio']
    available = [c for c in cols if c in growth_stocks.columns]
    display(growth_stocks[available].head(10))

## 5. Company Rankings

In [ ]:
# Initialize ranker
ranker = CompanyRanker(full_df)

# Calculate composite scores
rankings = ranker.calculate_composite_score()

print("\n🏆 TOP 10 STOCKS BY COMPOSITE SCORE:")
print("=" * 60)
print("(Combines Value, Growth, Dividend, Quality, Momentum, Safety)")
print()

display_cols = ['rank', 'symbol', 'name', 'composite_score', 
                'value_score', 'growth_score', 'dividend_score']
available = [c for c in display_cols if c in rankings.columns]
rankings[available].head(10)

In [ ]:
# Visualize top stocks
top_10 = rankings.head(10)

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(top_10['symbol'], top_10['composite_score'], color='steelblue')
ax.set_xlabel('Composite Score')
ax.set_title('🏆 Top 10 Stocks by Composite Score')
ax.invert_yaxis()  # Highest at top

# Add value labels
for bar, score in zip(bars, top_10['composite_score']):
    ax.text(bar.get_width() + 50, bar.get_y() + bar.get_height()/2, 
            f'{score:.0f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 6. Portfolio Suggestions

In [ ]:
# Portfolio suggestions
suggester = PortfolioSuggester(ranker)

print("\n📦 BALANCED PORTFOLIO SUGGESTION:")
print("=" * 50)
print("(Diversified across sectors and strategies)")
balanced = suggester.suggest_balanced_portfolio(10)
if not balanced.empty:
    cols = ['symbol', 'composite_score']
    if 'sector' in balanced.columns:
        cols.append('sector')
    display(balanced[[c for c in cols if c in balanced.columns]])

In [ ]:
print("\n💰 DIVIDEND INCOME PORTFOLIO:")
print("=" * 50)
income_portfolio = suggester.suggest_income_portfolio(5)
if not income_portfolio.empty:
    cols = ['symbol', 'name', 'dividend_yield']
    available = [c for c in cols if c in income_portfolio.columns]
    display(income_portfolio[available])

## 7. Custom Analysis

In [ ]:
# Custom screening - modify these criteria as needed
custom_criteria = {
    'pe_ratio': {'operator': 'lt', 'value': 12},
    'dividend_yield': {'operator': 'gt', 'value': 3},
    'roe': {'operator': 'gt', 'value': 12},
}

print("\n🔧 CUSTOM SCREEN RESULTS:")
print(f"Criteria: P/E < 12, Dividend Yield > 3%, ROE > 12%")
print("=" * 50)

custom_results = screener.screen_custom(custom_criteria)
print(f"Found {len(custom_results)} stocks matching criteria")

if not custom_results.empty:
    display(custom_results[['symbol', 'name', 'pe_ratio', 'dividend_yield', 'roe']].head(10))

In [ ]:
# Compare metrics scatter plot
if 'pe_ratio' in full_df.columns and 'dividend_yield' in full_df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    valid_df = full_df[(full_df['pe_ratio'] > 0) & (full_df['pe_ratio'] < 50)]
    
    scatter = ax.scatter(
        valid_df['pe_ratio'], 
        valid_df['dividend_yield'],
        c=valid_df.get('roe', 15),
        cmap='RdYlGn',
        s=100,
        alpha=0.7
    )
    
    # Add labels for top stocks
    for _, row in valid_df.head(5).iterrows():
        ax.annotate(row['symbol'], (row['pe_ratio'], row['dividend_yield']),
                   xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.colorbar(scatter, label='ROE (%)')
    ax.set_xlabel('P/E Ratio')
    ax.set_ylabel('Dividend Yield (%)')
    ax.set_title('P/E vs Dividend Yield (Color = ROE)')
    
    # Add quadrant lines
    ax.axvline(x=15, color='gray', linestyle='--', alpha=0.5)
    ax.axhline(y=4, color='gray', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

## 8. Export Results

In [ ]:
# Export to Excel
from reports.report_generator import ReportGenerator

reporter = ReportGenerator()
excel_path = reporter.generate_excel_report(
    full_df, 
    rankings, 
    strategy_results
)

print(f"✅ Report exported to: {excel_path}")

---
## 📚 Investment Strategy Guide

### Value Investing
- **P/E Ratio < 15**: Stock is cheap relative to earnings
- **P/B Ratio < 1.5**: Stock trades near or below book value
- Best for: Patient investors seeking undervalued companies

### Dividend Investing  
- **Dividend Yield > 4%**: High income generation
- **Payout Ratio < 70%**: Sustainable dividends
- Best for: Income-focused investors

### Growth Investing
- **ROE > 15%**: Efficient use of equity capital
- **EPS Growth > 10%**: Earnings growing consistently
- Best for: Long-term wealth builders

### Quality Investing
- **Low Debt/Equity**: Financially stable
- **Consistent margins**: Durable competitive advantage
- Best for: Risk-averse investors

---
⚠️ **Disclaimer**: This analysis is for educational purposes only. Always do your own research before investing.